In [8]:
search_for = 90
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004455089569091797
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 266587321
type: [1, 1, 1, 1, 90] 90
cases of this type: 65610000
10000 32.641203217691164
20000 39.729235090504154
30000 93.22573323568714
40000 152.02622429300487
50000 130.19314047213535
60000 157.47332596264192
70000 151.74384417279074
80000 165.0995912904671
90000 153.2319323303059
100000 158.5929050258741
110000 149.11793514676808
120000 155.61044362782758
130000 144.1998486614239
140000 152.0033817979543
150000 151.52779108949284
160000 153.93009996428503
170000 150.46973564752403
180000 162.4685195351791
190000 170.7373111637589
200000 165.43933060751277
210000 150.0662558427267
220000 156.46807513682504
230000 152.05513979927179
240000 149.6331178621659
250000 146.77232112482955
260000 147.25582662088064
270000 154.76375575993058
280000 145.14891776878574
290000 151.89258801510005
300000 148.59724030688812
310000 171.83

3050000 132.43551078818632
3060000 139.99230960398498
3070000 137.1814456603294
3080000 139.115350474002
3090000 150.90868958304597
3100000 147.85962560460555
3110000 141.54466027530395
3120000 142.78238250116038
3130000 145.33168789575495
3140000 138.8191119251635
3150000 132.67573067469198
3160000 138.29491291291
3170000 130.7372931767023
3180000 126.85553352433061
3190000 131.54525520580643
3200000 130.42352244665653
3210000 138.38841897792486
3220000 136.40682493085555
3230000 145.26299353075476
3240000 132.0667884287381
3250000 121.44958160291696
3260000 147.24950981029446
3270000 77.45669816657899
3280000 22.868603740567625
3290000 22.863865847189558
3300000 23.832910701978363
3310000 92.39484354869855
3320000 151.55305975027144
3330000 117.62200068036951
3340000 136.06848929945707
3350000 134.09302250102485
3360000 143.61113024513756
3370000 133.84384783778248
3380000 132.82626109483667
3390000 127.63498529167818
3400000 125.78650274877874
3410000 127.20700877288678
3420000 133.

6130000 127.67188615514696
6140000 125.62753244227022
6150000 123.36825175548964
6160000 115.6501995274357
6170000 118.00854729679406
6180000 141.2349615883398
6190000 26.403703174603702
6200000 22.146176202006977
6210000 22.225619435799775
6220000 22.162387676018636
6230000 140.60221140457602
6240000 128.3880767155279
6250000 105.69335357823013
6260000 126.14471343175362
6270000 126.20475106013826
6280000 124.399577921761
6290000 119.14168351148336
6300000 120.4678431412191
6310000 122.17434045094573
6320000 129.48587481492692
6330000 131.84902071895297
6340000 129.04709331283868
6350000 135.0370003331809
6360000 138.07548029691523
6370000 127.41661183543398
6380000 125.46544411144639
6390000 125.50474193230296
6400000 122.90967042269558
6410000 130.63581621946318
6420000 129.39715760818112
6430000 134.03235372651278
6440000 132.174637541594
6450000 126.97240070925388
6460000 126.64474432393436
6470000 134.42626441620436
6480000 143.90489252971417
6490000 53.3943333436949
6500000 54.5

9200000 113.18359654938467
9210000 112.784537864693
9220000 120.94326663201053
9230000 120.12872540567287
9240000 122.47988024926133
9250000 134.14446736510894
9260000 135.98810251512606
9270000 127.05380894964202
9280000 123.74933802967418
9290000 124.47236362911896
9300000 134.15279418365287
9310000 136.288664205217
9320000 134.93410451458826
9330000 135.06537752878327
9340000 137.7948771681831
9350000 130.90441499178132
9360000 134.60198187360285
9370000 88.67567151014505
9380000 53.34380121974737
9390000 53.95860247381739
9400000 54.13549075461072
9410000 117.10205717183864
9420000 139.49117985523614
9430000 125.10466383221208
9440000 135.65429409815397
9450000 128.15035389113
9460000 137.05057834421578
9470000 132.55367886902482
9480000 151.49320323500987
9490000 141.05666306716063
9500000 143.2067420440015
9510000 127.9655121080814
9520000 134.31788932759142
9530000 130.32933582702867
9540000 135.86228039518943
9550000 127.76242457129918
9560000 142.04046385792319
9570000 116.050

12190000 125.79814961496572
12200000 127.80774341487253
12210000 123.49152790864429
12220000 122.54968911381576
12230000 123.11169776531953
12240000 137.57582547505024
12250000 114.20327700644324
12260000 53.508450128623494
12270000 54.93612563928337
12280000 54.35089675924724
12290000 85.0394437974219
12300000 163.62620246427323
12310000 133.86906767823405
12320000 126.69782329902789
12330000 131.98067397760445
12340000 134.9978845509244
12350000 128.14534390398393
12360000 127.80697508792412
12370000 125.71129389623094
12380000 128.8539634471303
12390000 136.14035793290498
12400000 144.0833978493834
12410000 139.07517216248473
12420000 153.37459014259144
12430000 126.602562451016
12440000 131.24243384509433
12450000 139.72581124949852
12460000 126.85766068875415
12470000 136.33922305407077
12480000 141.3672091802697
12490000 138.00990249711887
12500000 130.07469728576652
12510000 160.29097385910845
12520000 95.23901348883106
12530000 44.10720238815981
12540000 44.18671703108542
12550

...

259600000 0.35840594703924655
259610000 0.2594004821851452
259620000 0.2602266634763678
259630000 0.36437654672116043
259640000 0.51154506089037
259650000 0.43484432426218855
259660000 0.4859796765080293
259670000 0.5403678299959169
259680000 0.5762216533104645
259690000 0.4127712065121532
259700000 0.26541581557348704
259710000 0.2713325611532993
259720000 0.3605050376083454
259730000 0.5064074965814219
259740000 0.42915701749930113
259750000 0.48993354725082716
259760000 0.47318092928755284
259770000 0.42282941761315535
259780000 0.41311232045406104
259790000 0.2551779038679203
259800000 0.2544590713721779
259810000 0.3179197373792529
259820000 0.4474248465322587
259830000 0.42059368000618613
259840000 0.4800061892418067
259850000 0.440113722180691
259860000 0.39776349950906303
259870000 0.42626721521764993
259880000 0.24143468317010666
259890000 0.24885235044425727
259900000 0.2966326539849838
259910000 0.3984743805870878
259920000 0.4393649817675154
259930000 0.4390463714180191
259

262270000 0.3004178602549831
262280000 0.2752533324217863
262290000 0.2697616047226853
262300000 0.2675256927255392
262310000 0.24802151118224855
262320000 0.25230733061079186
262330000 0.2537205800449848
262340000 0.2594474583019217
262350000 0.26332607428776555
262360000 0.2670503210915724
262370000 0.2861544482057426
262380000 0.2911255166852011
best so far: 0
type: [2, 1, 3, 3, 5] 90
cases of this type: 151875
262390000 0.278243276769042
262400000 0.17174697446718482
262410000 0.17163391260346442
262420000 0.17203130837855737
262430000 0.17006627863546214
262440000 0.17129465214137238
262450000 0.169326380939126
262460000 0.16989597903329134
262470000 0.16990203190535305
262480000 0.16781370013815164
262490000 0.16871772564917803
262500000 0.16841663613434632
262510000 0.16883517582160235
262520000 0.16765700287903015
262530000 0.1668828781073491
262540000 0.21672617914283274
best so far: 0
type: [2, 1, 3, 5, 3] 90
cases of this type: 91125
262550000 0.1667261115659674
262560000 0.

264260000 0.11830709762441449
264270000 0.18576712196701103
264280000 0.1442208567813635
264290000 0.15038372758707075
264300000 0.10588601272627382
264310000 0.0757881911097765
264320000 0.0763579848771029
264330000 0.0755076284861366
264340000 0.07485078233228128
264350000 0.07522336954489682
264360000 0.07455966266262531
264370000 0.07434268383729457
264380000 0.07571884964291253
264390000 0.07622972793403598
264400000 0.07585297992889087
264410000 0.16238819181980027
264420000 0.09890128341242341
264430000 0.17385139089415472
264440000 0.1251643505857057
264450000 0.1211292216433088
264460000 0.16058317359566687
264470000 0.10044911368822389
264480000 0.1531157299678458
264490000 0.15254556974514324
264500000 0.09315026096269158
264510000 0.16370836396910085
264520000 0.13774094155373176
264530000 0.11537587705355884
best so far: 0
type: [3, 1, 1, 3, 10] 90
cases of this type: 270000
264540000 0.14144364969555537
264550000 0.09923208083558083
264560000 0.09940865033970939
264570000

best so far: 0
type: [3, 3, 5, 2, 1] 90
cases of this type: 600
best so far: 0
type: [3, 3, 10, 1, 1] 90
cases of this type: 300
best so far: 0
type: [3, 5, 1, 1, 6] 90
cases of this type: 6480
266110000 0.01590910424033801
best so far: 0
type: [3, 5, 1, 2, 3] 90
cases of this type: 3240
best so far: 0
type: [3, 5, 1, 3, 2] 90
cases of this type: 2160
best so far: 0
type: [3, 5, 1, 6, 1] 90
cases of this type: 1080
best so far: 0
type: [3, 5, 2, 1, 3] 90
cases of this type: 1620
266120000 0.015149599505464234
best so far: 0
type: [3, 5, 2, 3, 1] 90
cases of this type: 540
best so far: 0
type: [3, 5, 3, 1, 2] 90
cases of this type: 720
best so far: 0
type: [3, 5, 3, 2, 1] 90
cases of this type: 360
best so far: 0
type: [3, 5, 6, 1, 1] 90
cases of this type: 180
best so far: 0
type: [3, 6, 1, 1, 5] 90
cases of this type: 3750
best so far: 0
type: [3, 6, 1, 5, 1] 90
cases of this type: 750
best so far: 0
type: [3, 6, 5, 1, 1] 90
cases of this type: 150
best so far: 0
type: [3, 10, 1, 1, 3

best so far: 0
type: [15, 1, 1, 2, 3] 90
cases of this type: 648
best so far: 0
type: [15, 1, 1, 3, 2] 90
cases of this type: 432
best so far: 0
type: [15, 1, 1, 6, 1] 90
cases of this type: 216
best so far: 0
type: [15, 1, 2, 1, 3] 90
cases of this type: 324
best so far: 0
type: [15, 1, 2, 3, 1] 90
cases of this type: 108
best so far: 0
type: [15, 1, 3, 1, 2] 90
cases of this type: 144
best so far: 0
type: [15, 1, 3, 2, 1] 90
cases of this type: 72
best so far: 0
type: [15, 1, 6, 1, 1] 90
cases of this type: 36
best so far: 0
type: [15, 2, 1, 1, 3] 90
cases of this type: 162
best so far: 0
type: [15, 2, 1, 3, 1] 90
cases of this type: 54
best so far: 0
type: [15, 2, 3, 1, 1] 90
cases of this type: 18
best so far: 0
type: [15, 3, 1, 1, 2] 90
cases of this type: 48
best so far: 0
type: [15, 3, 1, 2, 1] 90
cases of this type: 24
best so far: 0
type: [15, 3, 2, 1, 1] 90
cases of this type: 12
best so far: 0
type: [15, 6, 1, 1, 1] 90
cases of this type: 6
best so far: 0
type: [18, 1, 1, 1,